In [144]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

from sklearn.model_selection import train_test_split, cross_val_score

import lightgbm as lgb

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split, cross_val_predict
from sklearn.metrics import mean_squared_log_error

from sklearn.ensemble import VotingRegressor

from sklearn import linear_model

In [145]:
train  = pd.read_csv('../MachineHackE-Commerce/Train.csv')
test = pd.read_csv('../MachineHackE-Commerce/Test.csv')
sub = pd.read_excel('../MachineHackE-Commerce/Sample_Submission.xlsx')


In [506]:
test.tail()

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date
1046,P-9758,B-2543,sports fitness,outdoor adventure,camping hiking,2.0,28/2/2014
1047,P-11898,B-3197,jewellery,necklaces chains,necklaces,4.7,1/12/2019
1048,P-904,B-133,automotive,accessories spare parts,car interior exterior,3.5,1/12/2014
1049,P-1714,B-358,kitchen dining,lighting,bulbs,2.4,23/6/2013
1050,P-620,B-133,automotive,accessories spare parts,car interior exterior,3.1,1/12/2012


In [507]:
z =train[ train['Item_Category']=='automotive' ]

In [508]:
z[z['Subcategory_2']=='car interior exterior']

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date,Selling_Price
9,P-360,B-88,automotive,accessories spare parts,car interior exterior,2.3,10/5/2018,199.0
56,P-10422,B-2741,automotive,accessories spare parts,car interior exterior,3.4,21/4/2014,350.0
69,P-11595,B-3106,automotive,accessories spare parts,car interior exterior,3.5,24/5/2014,592.0
73,P-1004,B-133,automotive,accessories spare parts,car interior exterior,4.2,1/12/2017,1374.0
108,P-612,B-133,automotive,accessories spare parts,car interior exterior,3.9,1/12/2013,1348.0
...,...,...,...,...,...,...,...,...
2342,P-715,B-133,automotive,accessories spare parts,car interior exterior,2.6,1/12/2012,1594.0
2344,P-920,B-133,automotive,accessories spare parts,car interior exterior,4.3,1/12/2019,919.0
2347,P-975,B-133,automotive,accessories spare parts,car interior exterior,4.5,1/12/2019,1380.0
2436,P-829,B-133,automotive,accessories spare parts,car interior exterior,2.0,1/12/2017,1348.0


In [322]:
test

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date
0,P-11284,B-2984,computers,network components,routers,4.3,1/12/2018
1,P-6580,B-1732,jewellery,bangles bracelets armlets,bracelets,3.0,20/12/2012
2,P-5843,B-3078,clothing,women s clothing,western wear,1.5,1/12/2014
3,P-5334,B-1421,jewellery,necklaces chains,necklaces,3.9,1/12/2019
4,P-5586,B-3078,clothing,women s clothing,western wear,1.4,1/12/2017
...,...,...,...,...,...,...,...
1046,P-9758,B-2543,sports fitness,outdoor adventure,camping hiking,2.0,28/2/2014
1047,P-11898,B-3197,jewellery,necklaces chains,necklaces,4.7,1/12/2019
1048,P-904,B-133,automotive,accessories spare parts,car interior exterior,3.5,1/12/2014
1049,P-1714,B-358,kitchen dining,lighting,bulbs,2.4,23/6/2013


In [146]:
train_embedding = pd.read_csv('../MachineHackE-Commerce/train_embedding.csv')
train_topic = pd.read_csv('../MachineHackE-Commerce/train_topic.csv')
train_tfidf = pd.read_csv('../MachineHackE-Commerce/train_tfidf.csv')
train_enc = pd.read_csv('../MachineHackE-Commerce/train_enc.csv')

In [147]:
test_embedding = pd.read_csv('../MachineHackE-Commerce/test_embedding.csv')
test_topic = pd.read_csv('../MachineHackE-Commerce/test_topic.csv')
test_tfidf = pd.read_csv('../MachineHackE-Commerce/test_tfidf.csv')
test_enc = pd.read_csv('../MachineHackE-Commerce/test_enc.csv')

In [148]:
train_enc.columns

Index(['Product', 'Product_Brand', 'Item_Category', 'Subcategory_1',
       'Subcategory_2', 'Item_Rating', 'Selling_Price', 'Is_Holiday', 'Year',
       'Month', 'Week', 'Day', 'Dayofweek', 'Is_month_end', 'Is_month_start',
       'Is_quarter_end', 'Is_quarter_start'],
      dtype='object')

In [149]:
train_enc1 = train_enc.drop(['Product', 'Product_Brand', 'Item_Category', 'Subcategory_1',
       'Subcategory_2','Selling_Price'],axis=1)*1
test_enc1 = test_enc.drop(['Product', 'Product_Brand', 'Item_Category', 'Subcategory_1',
       'Subcategory_2'],axis=1)*1

In [150]:
def f_y(s):
    return s/1000

In [151]:
temptrain = f_y(train_enc1['Year'])
temptest = f_y(test_enc1['Year'])

In [152]:
train_enc1['Year'] = temptrain
test_enc1['Year'] = temptest

In [153]:
def f(s):
    return s/s.max()

In [154]:
train_enc1.columns

Index(['Item_Rating', 'Is_Holiday', 'Year', 'Month', 'Week', 'Day',
       'Dayofweek', 'Is_month_end', 'Is_month_start', 'Is_quarter_end',
       'Is_quarter_start'],
      dtype='object')

In [155]:
train_enc1['Month'] = train_enc1[['Month']].apply(f)
test_enc1['Month'] = test_enc1[['Month']].apply(f)


train_enc1['Week'] = train_enc1[['Week']].apply(f)
test_enc1['Week'] = test_enc1[['Week']].apply(f)

train_enc1['Day'] = train_enc1[['Day']].apply(f)
test_enc1['Day'] = test_enc1[['Day']].apply(f)

train_enc1['Dayofweek'] = train_enc1[['Dayofweek']].apply(f)
test_enc1['Dayofweek'] = test_enc1[['Dayofweek']].apply(f)


In [156]:
train_enc1.head()

,Item_Rating,Is_Holiday,Year,Month,Week,Day,Dayofweek,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start
0,4.3,0,2.017,0.166667,0.094340,0.096774,0.666667,0,0,0,0
1,3.1,0,2.015,0.583333,0.509434,0.032258,0.333333,0,1,0,1
2,3.5,1,2.019,0.083333,0.037736,0.387097,0.833333,0,0,0,0
3,4.0,0,2.014,1.000000,0.943396,0.387097,0.666667,0,0,0,0
4,4.3,0,2.013,1.000000,0.943396,0.387097,0.500000,0,0,0,0


In [157]:
encoded = pd.concat([train_enc1,test_enc1]).values
encoded.shape

(3503, 11)

In [159]:
train_topic.shape
topic = pd.concat([train_topic,test_topic]).values
topic.shape

(3503, 15)

In [161]:
NUM_BRANDS = 790
NUM_CATEGORIES = 1001
NAME_MIN_DF = 10
MAX_FEATURES_ITEM_DESCRIPTION = 3

In [162]:
nrow_train = train.shape[0]
y = np.log1p(train["Selling_Price"])
#y = train["Selling_Price"]
dataset = pd.concat([train, test])
      

/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [163]:
dataset['Item_Category'] = dataset['Item_Category'].astype('category')
dataset['Product_Brand'] = dataset['Product_Brand'].astype('category')
dataset['Subcategory_1'] = dataset['Subcategory_1'].astype('category')
dataset['Subcategory_2'] = dataset['Subcategory_2'].astype('category')

dataset['text'] = dataset.apply(lambda row: ' '.join ([str(row['Item_Category']), str(row['Subcategory_1']),str(row['Subcategory_2'])]), 
                                          axis=1)



In [164]:
dataset.head()

,Date,Item_Category,Item_Rating,Product,Product_Brand,Selling_Price,Subcategory_1,Subcategory_2,text
0,2/3/2017,bags wallets belts,4.3,P-2610,B-659,291.0,bags,hand bags,bags wallets belts bags hand bags
1,7/1/2015,clothing,3.1,P-2453,B-3078,897.0,women s clothing,western wear,clothing women s clothing western wear
2,1/12/2019,home decor festive needs,3.5,P-6802,B-1810,792.0,showpieces,ethnic,home decor festive needs showpieces ethnic
3,12/12/2014,beauty and personal care,4.0,P-4452,B-3078,837.0,eye care,h2o plus eye care,beauty and personal care eye care h2o plus eye...
4,12/12/2013,clothing,4.3,P-8454,B-3078,470.0,men s clothing,t shirts,clothing men s clothing t shirts


In [165]:
cv = CountVectorizer(min_df=NAME_MIN_DF)
X_Item_Category = cv.fit_transform(dataset['Item_Category'])
    

In [166]:
cv = CountVectorizer()
X_category1 = cv.fit_transform(dataset['Subcategory_1'])
    

In [167]:
cv2 = CountVectorizer()
X_category2 = cv.fit_transform(dataset['Subcategory_2'])
    

In [168]:
tv = TfidfVectorizer(max_features=MAX_FEATURES_ITEM_DESCRIPTION,
                         ngram_range=(1, 3),
                         stop_words='english')
X_description = tv.fit_transform(dataset['text'])
    

In [169]:
lb = LabelBinarizer(sparse_output=True)
X_brand = lb.fit_transform(dataset['Product_Brand'])
    

In [170]:
dataset['Product'] = dataset['Product'].str.replace(r'\D', '')

dataset['Product'] = pd.to_numeric(dataset['Product'], errors='coerce')


In [171]:
X_product = f(dataset['Product'])

In [172]:
X_prod=np.array(X_product).reshape(-1,1)

In [173]:
X_description.shape
X_category1.shape
X_category2.shape
X_Item_Category.shape
X_prod.shape

(3503, 1)

In [ ]:
X_description, X_brand, X_category1,X_category2, X_Item_Category , X_prod,encoded

In [761]:
sparse_merge1 = hstack((X_prod,X_brand,X_Item_Category ,X_category1,X_category2,X_description,encoded)).tocsr()
X1 = sparse_merge1[:nrow_train]
X_test1 = sparse_merge1[nrow_train:]
   

In [762]:
# sparse_merge3 = hstack((X_description, X_brand, X_category1,X_category2, X_Item_Category , X_prod,encoded,topic)).tocsr()
# X3 = sparse_merge3[:nrow_train]
# X_test3 = sparse_merge3[nrow_train:]
   

In [177]:
# def rmsle_cv(model,X,y):
#     n_folds = 5
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X)
#     rmsle= np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_log_error", cv = kf))
#     print(rmsle.mean())
#     return(rmsle)

In [178]:
# def rmsle_cv_ensemble(model,X,y):
#     n_folds = 5
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X)
    
#     y_pred = cross_val_predict(model, X, np.log1p(y), cv=kf)
#     if (y_pred<0).sum() :
#             print("negtive {0} value predicted".format( (y_pred<0).sum())) 
          
        
#     y_pred_inverse = np.expm1(y_pred)
    
    
#     score = np.sqrt(mean_squared_log_error(y, y_pred))
    
#     return(score)

In [ ]:
X3.shape

In [179]:
def rmsle(y, y_pred):
        return np.sqrt(mean_squared_log_error(y, y_pred))

In [384]:
# d_train = lgb.Dataset(X, label= y)
# params = {
#         'learning_rate': 0.45,
#         'application': 'regression',
#         'max_depth': 10,
#         'num_leaves': 1000,
#         'verbosity': -1,
#         'metric': 'RMSLE'}

# model_lgb = lgb.train(params, train_set=d_train, num_boost_round=3200, verbose_eval=100) 

# model_lgb_train_pred = np.expm1(model_lgb.predict(X))

# model_lgb_pred = np.expm1(model_lgb.predict(X_test))



In [385]:
# model_lgb_pred

In [386]:
# print(rmsle(np.expm1(y), model_lgb_train_pred))

In [763]:
model_ridge =Ridge(solver="sag", fit_intercept=True, random_state=2)

model_ridge.fit(X3, y)

model_ridge_train_pred = np.expm1(model_ridge.predict(X3))

model_ridge_pred = np.expm1(model_ridge.predict(X_test3))

print(rmsle(np.expm1(y), model_ridge_train_pred))

0.3813047020208203


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:558: UserWarning: "sag" solver requires many iterations to fit an intercept with sparse inputs. Either set the solver to "auto" or "sparse_cg", or set a low "tol" and a high "max_iter" (especially if inputs are not standardized).
  '"sag" solver requires many iterations to fit '


In [764]:
model_ridge_pred

array([2042.39230208,  526.17707262,  639.14506068, ..., 1349.13640926,
        380.9456279 , 1466.29523668])

In [373]:
model_ridge_pred

array([2042.39230208,  526.17707262,  639.14506068, ..., 1349.13640926,
        380.9456279 , 1466.29523668])

In [387]:
from sklearn.model_selection import GridSearchCV

In [755]:
# from sklearn.model_selection import RandomizedSearchCV
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 50, stop = 500, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(11, 33, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

In [756]:
# rf = RandomForestRegressor()
# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# # Fit the random search model
# rf_random.fit(X1, y)

In [757]:
# rf_random.best_params_

In [753]:
model_rf = RandomForestRegressor(n_estimators= 100,
 min_samples_split= 2,
 min_samples_leaf =1,
 max_features= 'sqrt',
 max_depth= 23,
 bootstrap= False,random_state=2)

model_rf.fit(X1, y)
model_rf_train_pred = np.expm1(model_rf.predict(X1))

model_rf_pred = np.expm1(model_rf.predict(X_test1))
print(rmsle(np.expm1(y), model_rf_train_pred))

0.4715580313516514


In [754]:
model_rf_pred

array([1954.90909907,  570.70547988,  696.45048924, ..., 1058.98826441,
        555.8684146 , 1383.58538592])

In [752]:
model_rf_pred

array([1954.90909907,  570.70547988,  696.45048924, ..., 1058.98826441,
        555.8684146 , 1383.58538592])

In [745]:

print('RMSLE score on train data:')
print(rmsle(np.expm1(y), model_rf_train_pred*0.24 + model_ridge_train_pred*0.76))

RMSLE score on train data:
0.38363903214529416


In [748]:
ensemble =  model_rf_pred*0.24 + model_ridge_pred*0.76 

In [749]:
ensemble

array([2021.39633335,  536.86389036,  652.89836354, ..., 1279.50085449,
        422.9270967 , 1446.4448725 ])

In [751]:
sub['Selling_Price'] = ensemble

sub.to_excel("submission_rf_ridge_X2final.xlsx", index=False)
